# Análisis de las señales usando inferencia bayesiana

In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import interpolate
import seaborn as sns
import pandas as pd

import theano.tensor as TT
import time
import random

import pymc3 as pm
import arviz as az

%run /home/tele/Desktop/UWB/codigos_auxiliares/Bayes.ipynb

ModuleNotFoundError: No module named 'theano'

In [ ]:

portadoras = ['Fc7/','Fc8/']
portadora = portadoras[1]

alturas = ['1/','2/','3/','4/']
altura = alturas[2]

In [ ]:
%run /home/tele/Desktop/UWB/mediciones_09_06/ajustes_09_06_Caro.ipynb

## Cargo las mediciones

In [ ]:
## Señal de Calibración
Scal = np.load('Scal.npy',mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
print('dim Scal:',np.shape(Scal)[0])
print()

## Señal del target
Star = -np.load('Star.npy',mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
print('dim Star:',np.shape(Star)[0])
print()

## Frecuencias positivas obtenidas de la FFT
fpos = np.load('fpos.npy',mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
print('dim fpos:',np.shape(fpos)[0])

## Skin Depth de la señal UWB

sea $\epsilon_1=\epsilon_r+i\epsilon_i$ entonces defino:

\begin{equation}
    \delta_d=\frac{2\epsilon_r 2\times10^8}{2\pi f \epsilon_i}
\end{equation}


OBSERVACION: Puede que este mal ese $2\times10^8$ y que sea $3\times10^8=c$?


In [ ]:
ep1_obs = 11.340 + 1j*4.104  # dieléctrico

## skin depth para la portadora con ep1 medido
landa = 3e8/fc
print('long. onda portadora:',str(round(100*landa,2))+'cm')
delta_d = 2*np.real(ep1_obs)*2e8/(2*np.pi*(fc)*np.imag(ep1_obs))
print('Skin Depth a Fc:', str(round(100*delta_d,2))+'cm')
print()
print()

landa_max = 3e8/(fc-BW)
print('long. onda max:',str(round(100*landa_max,2))+'cm')
delta_max = 2*np.real(ep1_obs)*2e8/(2*np.pi*(fc-BW)*np.imag(ep1_obs))
print('Skin Depth a long. onda max:', str(round(100*delta_max,2))+'cm')
print()

## Simulo una medición y busco los parámetros óptimos a través del esquema bayesiano.

## - Un solo dieléctrico

In [ ]:
# inputs para simular una medición

# un solo medio dieléctrico
NLayers = 1

S_sim = Scal*R_ep(ep1_obs) # señal 'medida'

# genero señal con ruido
S_sim_ruido = np.zeros(len(S_sim), dtype = complex)
noise_level = 1.2e-2
noise = (1+1j)*noise_level*np.random.rand()

for i in range(len(S_sim)):
    S_sim_ruido[i] =  S_sim[i] + random.choice((-1,1))*noise


fig , ax = plt.subplots(nrows=1, ncols=2, figsize=(16,6))

ax[0].plot(fpos/1e9,np.real(S_sim),label="teorica")
ax[0].plot(fpos/1e9,np.real(S_sim_ruido),label="con ruido")
ax[0].plot(fpos/1e9,np.real(Star),'-.',label="medicion")
ax[0].legend(loc="best")
ax[0].set_title('Parte real')

ax[1].plot(fpos/1e9,np.imag(S_sim),label="teorica")
ax[1].plot(fpos/1e9,np.imag(S_sim_ruido),label="con ruido")
ax[1].plot(fpos/1e9,np.imag(Star),'-.',label="medicion")
ax[1].legend(loc="best")
ax[1].set_title('Parte imaginaria')
plt.show()

print()
print('nivel de ruido:', round(np.real(noise),4))

In [ ]:
%run /home/tele/Desktop/UWB/codigos_auxiliares/Bayes.ipynb

t0 = time.time()
# nro de muestras que va a utilizar el método
muestras = 8000
# error de la medición
sigmas = np.real(noise)*np.ones((2,len(Star)))


# inferencia bayesiana
#traza_sim_1layer, ppc, model = modelo(-Star,muestras,sigmas,NLayers)
traza_1layer = modelo(Star,muestras,sigmas,NLayers)


t1 = time.time()
print('Tiempo insumido - Inf. Bayesiana: '+str(round(t1-t0,2))+' seg', '('+str(round((t1-t0)/60,2))+' min)')

In [ ]:
traza_1layer['ep1_r'].shape

In [ ]:
t2 = time.time()
print('Graficando con un solo dieléctrico ...')

ep1_r_mean = round(np.mean(traza_1layer['ep1_r']),3)
ep1_r_std = round(np.std(traza_1layer['ep1_r']),3)

ep1_i_mean = round(np.mean(traza_1layer['ep1_i']),3)
ep1_i_std = round(np.std(traza_1layer['ep1_i']),3)

# plotear = np.stack((traza['ep1'], traza['ep2'], traza['d'], traza['s1'], traza['s2'])).T 
# dfplotear = pd.DataFrame(plotear, columns = ['ep1','ep2','d','s1','s2'])

plotear = np.stack((traza_1layer['ep1_r'], traza_1layer['ep1_i'])).T 
dfplotear = pd.DataFrame(plotear, columns = ['ep1_r', 'ep1_i'])


g = sns.PairGrid(dfplotear)
g.fig.suptitle('Medición sintética con un solo medio material', y=1.01, fontsize = 'large')
g.map_upper(sns.histplot)
g.map_lower(sns.kdeplot, fill=True)
g.map_diag(sns.histplot, kde=True, color='.5')
# g.axes[0,0].axvline(np.real(ep1_obs), color='r', linestyle='solid', linewidth=0.5)
# g.axes[1,0].scatter(np.real(ep1_obs), np.imag(ep1_obs), marker="+", c='r', s=200)
# g.axes[1,1].axvline(np.imag(ep1_obs), color='r', linestyle='solid', linewidth=0.5)


print()
print('input ep1_r: ', np.real(ep1_obs))
print('output ep1_r: ', str(ep1_r_mean)+"\u00B1"+str(ep1_r_std))
print()
print('input ep1_i: ', np.imag(ep1_obs))
print('output ep1_i: ', str(ep1_i_mean)+"\u00B1"+str(ep1_i_std))
print()

t3 = time.time()
print('Tiempo insumido - Gráficos: '+str(round(t3-t2,2))+' seg', '('+str(round((t3-t2)/60,2))+' min)')
print()

print('Tiempo insumido - TOTAL: '+str(round(t3-t0,2))+' seg', '('+str(round((t3-t0)/60,2))+' min)')
print()
print()
print()

In [ ]:
EP1 = ep1_r_mean + 1j*ep1_i_mean
S_fit = Scal*R_ep(EP1)

fig , ax = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

ax[0].plot(fpos/1e9,np.real(Star),'-.',label="Medicion")
ax[0].plot(fpos/1e9,np.real(S_fit),label="Ajuste")
ax[0].grid()
ax[0].set_xlabel('Frec [GHz]')
ax[0].set_title('Parte Real')
ax[0].legend(loc="best")

ax[1].plot(fpos/1e9,np.imag(Star),'-.',label="Medicion")
ax[1].plot(fpos/1e9,np.imag(S_fit),label="Ajuste")
ax[1].grid()
ax[1].set_xlabel('Frec [GHz]')
ax[1].set_title('Parte Imag.')
ax[1].legend(loc="best")

ax[2].plot(fpos/1e9,np.abs(Star),'-.',label="Medicion")
ax[2].plot(fpos/1e9,np.abs(S_fit),label="Ajuste")
ax[2].grid()
ax[2].set_xlabel('Frec [GHz]')
ax[2].set_title('Módulo')
ax[2].legend(loc="best")
plt.show()